<a href="https://colab.research.google.com/github/gotjd709/cv_model_torch/blob/master/classification_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ResNet


## 1) No Residual Learning (Plain Network)

### 1-1) PlainBlock 
=> 18 layers, 34 layers
- Whenever the layer name is changed, the number of input planes of the first block chages. 
- The exception is that the first layer name is changed.

### 1-2) BottlePlain 
=> 52 layers, 101 layers, 152 layers
- It is the same as in PlainBlock, but there is a 1x1 convolution, so more convolution layers should be considered.

In [37]:
import torch.nn as nn
import torch

def conv1x1(ic, oc, s):
    return nn.Conv2d(ic, oc, 1, s)

def conv3x3(ic, oc, s, p):
    return nn.Conv2d(ic, oc, 3, s, p) 

class PlainBlock(nn.Module):
    def __init__(self, ln, bn, fn, s, p):
        super().__init__()
        self.conv1 = conv3x3(fn*(2**(ln-1)), fn*(2**(ln)), s*2, p) if ln != 0 else conv3x3(fn*(2**(ln)), fn*(2**(ln)), s*2, p)
        self.conv2 = conv3x3(fn*(2**(ln)), fn*(2**(ln)), s, p)
        self.batch = nn.BatchNorm2d(fn*(2**ln))
        self.relu  = nn.ReLU()
        self.ln = ln
        self.bn = bn

    def forward(self, x):
        if (self.ln != 0) and (self.bn == 1):
            conv1  = self.conv1(x)
        else:
            conv1  = self.conv2(x)
        batch1 = self.batch(conv1)
        relu1  = self.relu(batch1)
        conv2  = self.conv2(relu1)
        batch2 = self.batch(conv2)
        relu2  = self.relu(batch2)
        return relu2

class BottlePlain(nn.Module):
    def __init__(self, ln, bn, fn, s, p):
        super().__init__()
        self.conv11= conv1x1(fn*(2**(ln)), fn*(2**(ln)), s)
        self.conv12= conv1x1(fn*(2**(ln)*2), fn*(2**(ln)), s) 
        self.conv13= conv1x1(fn*(2**(ln)*4), fn*(2**(ln)), s)
        self.conv2 = conv3x3(fn*(2**(ln)), fn*(2**(ln)), s*2, p) if (ln != 0) and (bn == 1) else conv3x3(fn*(2**(ln)), fn*(2**(ln)), s, p)
        self.conv3 = conv1x1(fn*(2**(ln)), fn*(2**(ln))*4, s)
        self.batch = nn.BatchNorm2d(fn*(2**ln))
        self.batch2= nn.BatchNorm2d(fn*(2**ln)*2)
        self.batch4= nn.BatchNorm2d(fn*(2**ln)*4)
        self.relu  = nn.ReLU()
        self.ln = ln
        self.bn = bn

    def forward(self, x):
        if (self.ln == 0) and (self.bn == 1):
            conv1 = self.conv11(x)
        elif (self.ln != 0) and (self.bn == 1):
            conv1 = self.conv12(x)
        else:
            conv1 = self.conv13(x)
        batch1 = self.batch(conv1)
        relu1  = self.relu(batch1)
        conv2  = self.conv2(relu1)
        batch2 = self.batch(conv2)
        relu2  = self.relu(batch2)
        conv3  = self.conv3(relu2)
        batch4 = self.batch4(conv3)
        relu3  = self.relu(batch4)
        return relu3


class PlainNet(nn.Module):
    '''
    PlainNet is ResNet without residual learning.

    Args:
        btype: Type of the block that will be used as an residual learning.
            Available option: **PlainBlock** (for PlainNet18, PlainNet34) and **BottlePlain(Plain50, PlainNet101, PlainNet152)**
        block: List of the block number. (e.g PlainNet34 -> [3,4,6,3])
        fn: Number of features. default is 64.
        s: stride.
        p: padding.
        classes: number of classes.
    '''
    def __init__(self, btype, block, fn, s, p, classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3,fn,7,2,3)
        self.batch = nn.BatchNorm2d(fn)
        self.relu = nn.ReLU()
        self.maxp1 = nn.MaxPool2d(2,2,0)
        self.block1 = self._make_layer(btype, 0, block[0], fn, s, p)
        self.block2 = self._make_layer(btype, 1, block[1], fn, s, p)
        self.block3 = self._make_layer(btype, 2, block[2], fn, s, p)
        self.block4 = self._make_layer(btype, 3, block[3], fn, s, p)
        self.aapool = nn.AdaptiveAvgPool2d((1,1))
        self.flaten = nn.Flatten()
        self.adjust = 1 if btype == 'PlainBlock' else 4
        self.linear = nn.Linear(in_features=(fn*2**(len(block)-1)*self.adjust),out_features=classes)

    def _make_layer(self, btype, ln, block, fn, s, p):
        layers = []
        for bn in range(1,block+1):
            layers.append(PlainBlock(ln, bn, fn, s, p)) if btype == 'PlainBlock' else layers.append(BottlePlain(ln, bn, fn, s, p))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch(x)
        x = self.relu(x)
        x = self.maxp1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.aapool(x)
        x = self.flaten(x)
        x = self.linear(x)
        return x

def plain18(btype='PlainBlock', block=[2,2,2,2], fn=64, s=1, p=1, classes=1000):
    return PlainNet(btype, block, fn, s, p, classes)

def plain34(btype='PlainBlock', block=[3,4,6,3], fn=64, s=1, p=1, classes=1000):
    return PlainNet(btype, block, fn, s, p, classes)

def plain50(btype='BottlePlain', block=[3,4,6,3], fn=64, s=1, p=1, classes=1000):
    return PlainNet(btype, block, fn, s, p, classes)

def plain101(btype='BottlePlain', block=[3,4,23,3], fn=64, s=1, p=1, classes=1000):
    return PlainNet(btype, block, fn, s, p, classes)

def plain152(btype='BottlePlain', block=[3,8,36,3], fn=64, s=1, p=1, classes=1000):
    return PlainNet(btype, block, fn, s, p, classes)

## 2) ResNet

### 2-1) BasicBlock 
=> 18 layers, 34 layers
- Whenever the layer name is changed, the number of input planes of the first block chages. 
- The exception is that the first layer name is changed.
- x residual should be same shape of a prior layer shape.

### 2-2) Bottleneck 
=> 52 layers, 101 layers, 152 layers
- It is the same as in BasicBlock, but there is a 1x1 convolution, so more convolution layers should be considered.

In [110]:
import torch.nn as nn
import torch

def conv1x1(ic, oc, s):
    return nn.Conv2d(ic, oc, 1, s)

def conv3x3(ic, oc, s, p):
    return nn.Conv2d(ic, oc, 3, s, p) 

class BasicBlock(nn.Module):
    def __init__(self, ln, bn, fn, s, p):
        super().__init__()
        self.conv1 = conv3x3(fn*(2**(ln-1)), fn*(2**(ln)), s*2, p) if ln != 0 else conv3x3(fn*(2**(ln)), fn*(2**(ln)), s*2, p)  # just to prevent error     
        self.conv2 = conv3x3(fn*(2**(ln)), fn*(2**(ln)), s, p)
        self.skip  = nn.Sequential(
            conv1x1(fn*(2**(ln-1)), fn*(2**(ln)), s*2) if ln != 0 else conv1x1(fn*(2**(ln)), fn*(2**(ln)), s*2),
            nn.BatchNorm2d(fn*(2**(ln)))
        )   
        self.batch = nn.BatchNorm2d(fn*(2**ln))
        self.relu  = nn.ReLU()
        self.ln = ln
        self.bn = bn

    def forward(self, x):
        if (self.ln != 0) and (self.bn == 1):
            conv1 = self.conv1(x)
        else:
            conv1 = self.conv2(x)
        batch1 = self.batch(conv1)
        relu1  = self.relu(batch1)
        conv2  = self.conv2(relu1)
        batch2 = self.batch(conv2)
        if (self.ln != 0) and (self.bn == 1):
            add = torch.add(self.skip(x), batch2)
        else:
            add = torch.add(x, batch2)
        relu2  = self.relu(add)
        return relu2

class Bottleneck(nn.Module):
    def __init__(self, ln, bn, fn, s, p):
        super().__init__()
        self.conv11= conv1x1(fn*(2**(ln)), fn*(2**(ln)), s)
        self.conv12= conv1x1(fn*(2**(ln)*2), fn*(2**(ln)), s) 
        self.conv13= conv1x1(fn*(2**(ln)*4), fn*(2**(ln)), s)
        self.conv2 = conv3x3(fn*(2**(ln)), fn*(2**(ln)), s*2, p) if (ln != 0) and (bn == 1) else conv3x3(fn*(2**(ln)), fn*(2**(ln)), s, p)
        self.conv3 = conv1x1(fn*(2**(ln)), fn*(2**(ln))*4, s)
        self.skip1 = nn.Sequential(
            conv1x1(fn*(2**(ln)), fn*(2**(ln)*4), s),
            nn.BatchNorm2d(fn*(2**ln)*4)                     
        )
        self.skip2 = nn.Sequential(
            conv1x1(fn*(2**(ln)*2), fn*(2**(ln)*4), s*2),
            nn.BatchNorm2d(fn*(2**ln)*4)                     
        )
        self.batch = nn.BatchNorm2d(fn*(2**ln))
        self.batch4= nn.BatchNorm2d(fn*(2**ln)*4)
        self.relu  = nn.ReLU()
        self.ln = ln
        self.bn = bn

    def forward(self, x):
        if (self.ln == 0) and (self.bn == 1):
            conv1 = self.conv11(x)
        elif (self.ln != 0) and (self.bn == 1):
            conv1 = self.conv12(x)
        else:
            conv1 = self.conv13(x)
        batch1 = self.batch(conv1)
        relu1  = self.relu(batch1)
        conv2  = self.conv2(relu1)
        batch2 = self.batch(conv2)
        relu2  = self.relu(batch2)
        conv3  = self.conv3(relu2)
        batch4 = self.batch4(conv3)
        if (self.ln == 0) and (self.bn == 1):
            add = torch.add(self.skip1(x), batch4)
        elif (self.ln != 0) and (self.bn == 1):
            add = torch.add(self.skip2(x), batch4)
        else:
            add = torch.add(x, batch4)
        relu3  = self.relu(add)
        return relu3

class ResNet(nn.Module):
    '''
    ResNet is fully convolution neural network for image classification.

    Args:
        btype: Type of the block that will be used as an residual learning.
            Available option: **BasicBlock** (for ResNet18, ResNet34) and **Bottleneck(ResNet50, ResNet101, ResNet152)**
        block: List of the block number. (e.g ResNet34 -> [3,4,6,3])
        fn: Number of features. default is 64.
        s: stride.
        p: padding.
        classes: number of classes.
    '''
    def __init__(self, btype, block, fn, s, p, classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3,fn,7,2,3)
        self.batch = nn.BatchNorm2d(fn)
        self.relu = nn.ReLU()
        self.maxp1 = nn.MaxPool2d(2,2,0)
        self.block1 = self._make_layer(btype, 0, block[0], fn, s, p)
        self.block2 = self._make_layer(btype, 1, block[1], fn, s, p)
        self.block3 = self._make_layer(btype, 2, block[2], fn, s, p)
        self.block4 = self._make_layer(btype, 3, block[3], fn, s, p)
        self.aapool = nn.AdaptiveAvgPool2d((1,1))
        self.flaten = nn.Flatten()
        self.adjust = 1 if btype == 'BasicBlock' else 4
        self.linear = nn.Linear(in_features=(fn*2**(len(block)-1)*self.adjust),out_features=classes)

    def _make_layer(self, btype, ln, block, fn, s, p):
        layers = []
        for bn in range(1,block+1):
            layers.append(BasicBlock(ln, bn, fn, s, p)) if btype == 'BasicBlock' else layers.append(Bottleneck(ln, bn, fn, s, p))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch(x)
        x = self.relu(x)
        x = self.maxp1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.aapool(x)
        x = self.flaten(x)
        x = self.linear(x)
        return x

def resnet18(btype='BasicBlock', block=[2,2,2,2], fn=64, s=1, p=1, classes=1000):
    return ResNet(btype, block, fn, s, p, classes)

def resnet34(btype='BasicBlock', block=[3,4,6,3], fn=64, s=1, p=1, classes=1000):
    return ResNet(btype, block, fn, s, p, classes)

def resnet50(btype='Bottleneck', block=[3,4,6,3], fn=64, s=1, p=1, classes=1000):
    return ResNet(btype, block, fn, s, p, classes)

def resnet101(btype='Bottleneck', block=[3,4,23,3], fn=64, s=1, p=1, classes=1000):
    return ResNet(btype, block, fn, s, p, classes)

def resnet152(btype='Bottleneck', block=[3,8,36,3], fn=64, s=1, p=1, classes=1000):
    return ResNet(btype, block, fn, s, p, classes)